In [50]:
# We will use a softmax classifer. First we need a package. 

import torch.nn.functional as F


In [54]:
# Now we use the softmax for our model. 

probs = F.softmax(outputs, dim = 1)

# Lets look at an example of it being applied. 

print('Some examples : \n', probs[:2].data)

# Add the prob for a output row. 

print("Sum :", torch.sum(probs[0]).item())

Some examples : 
 tensor([[0.0989, 0.1288, 0.1351, 0.1028, 0.0933, 0.0710, 0.1120, 0.0852, 0.0828,
         0.0901],
        [0.0893, 0.1134, 0.1106, 0.1242, 0.0845, 0.0920, 0.1319, 0.0753, 0.0812,
         0.0976]])
Sum : 1.0000001192092896


In [56]:
# Now we need to take the largest probablity and associate it with a label from 0 to 9 

max_probs, preds = torch.max(probs, dim = 1)
print(preds)
print(max_probs)

tensor([2, 6, 6, 6, 6, 2, 6, 2, 6, 9, 2, 6, 6, 2, 1, 5, 6, 1, 6, 2, 6, 6, 2, 2,
        2, 1, 1, 2, 6, 1, 1, 6, 2, 6, 2, 1, 2, 2, 8, 2, 2, 6, 2, 3, 2, 2, 6, 6,
        6, 2, 6, 6, 6, 1, 6, 6, 2, 6, 1, 6, 6, 2, 3, 8, 6, 3, 0, 1, 6, 1, 2, 2,
        2, 1, 6, 2, 6, 2, 6, 6, 2, 1, 5, 6, 2, 8, 6, 6, 0, 6, 2, 2, 2, 3, 6, 2,
        9, 6, 2, 2, 2, 1, 6, 1, 6, 9, 3, 6, 2, 6, 1, 1, 3, 3, 6, 6, 1, 6, 1, 1,
        2, 2, 6, 6, 2, 6, 6, 6])
tensor([0.1351, 0.1319, 0.1234, 0.1483, 0.1518, 0.1363, 0.1259, 0.1329, 0.1347,
        0.1166, 0.1485, 0.1650, 0.1348, 0.1558, 0.1265, 0.1228, 0.1243, 0.1297,
        0.1326, 0.1257, 0.1261, 0.1513, 0.1407, 0.1432, 0.1532, 0.1351, 0.1286,
        0.1561, 0.1321, 0.1239, 0.1420, 0.1373, 0.1227, 0.1277, 0.1537, 0.1408,
        0.1303, 0.1307, 0.1484, 0.1445, 0.1384, 0.1385, 0.1503, 0.1284, 0.1797,
        0.1285, 0.1171, 0.1281, 0.1593, 0.1505, 0.1303, 0.1246, 0.1391, 0.1338,
        0.1468, 0.1371, 0.1507, 0.1223, 0.1472, 0.1554, 0.1264, 0.1325, 0.1265,
       

In [58]:
# Define an accuracy for our model and the pred. 

def acc(outputs,labels):
    probability, preds = torch.max(outputs, dim = 1)
    return torch.tensor(torch.sum(preds == labels).item()/ len(preds))

In [59]:
# The model with random Weights and Bias has an accuracy of 9%. 

acc(outputs,labels)

tensor(0.0938)

In [60]:
# We will use crossentropy as our loss function to outdate our model. 

loss_fn = F.cross_entropy

In [61]:
# Apply it to the data we have. 

loss = loss_fn(outputs,labels)
print(loss)

tensor(2.3112, grad_fn=<NllLossBackward>)


In [63]:
# Lets now create the model and all the moving parts to it. 

def fit(epochs, lr, model, train_loader,val_loader, opt_func = torch.optim.SGD):
    optimizer = opt_func(model.parameters(),lr)
    history = [] # recording the results 
    
    for epoch in range(epochs):
        
        #Training
        for batch in train_loader: 
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            
        #Valid
        
        result = evaluate(model, val_loader)
        model.epoch_end(epoch,result)
        history.append(result)
        
    return history 

In [64]:
# We create the evaluate function. 

def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [77]:
# Now we create our final Mnistmodel that we will test. 

class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.Linear = nn.Linear(input_size,num_classes)
        
    def forward(self,xb):
        xb = xb.reshape(-1,784)
        out = self.Linear(xb)
        return out
    
    def training_step(self,batch):
        images, labels = batch
        out = self(batch)
        loss = F.cross_entropy(out,labels)
        return loss 
    
    def validation_step(self,batch):
        images, labels = batch 
        out = self(images)
        loss = F.cross_entropy(out, labels)
        accu = acc(out, labels)
        return {'val_loss': loss, 'val_acc': accu}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_losses = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_losses.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self,epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch,result['val_loss'], result['val_acc']))
    
model = MnistModel()

In [78]:
# Try the function on a sample set 

results1 = evaluate(model,val_loader)
results1

{'val_loss': 2.284787893295288, 'val_acc': 0.1461629718542099}